In [1]:
import pandas as pd
from NLP_pp import *

In [2]:
raw_data = pd.read_csv('exam_deko.csv')

display(raw_data)

,문서ID,번역,원문(src),번역문(tar)
0,ot_100615,독->한,"mal sehen , ob meine haut reagiert oder nicht .",내 피부가 반응하는지 보자 .
1,ot_100616,독->한,ich habe viele großartige kritiken über drunk ...,저는 드렁크 엘리펀트에 대한 훌륭한 리뷰를 많이 들었습니다 .
2,ot_100617,독->한,lass uns zur poolparty gehen .,수영장 파티에 가자 .
3,ot_100618,독->한,sie bringen einen neuen reiniger auf den markt .,그들은 새로운 청소기를 시장에서 출시하고 있어요 .
4,ot_100619,독->한,genau hier ist nur gepresster gemüsesaft .,바로 여기에 압착 야채 주스가 있어요 .
...,...,...,...,...
120327,md_100608,독->한,"er sucht nach dem ort , der die bewohner verei...","그는 주민들을 결집시킬 수 있는 장소를 찾아다녔고 , 이를 빵집에서 찾는다 ."
120328,md_100609,독->한,"coelho costinha ist ein einfacher mann , der j...",코엘호 코스틴하는 재정적인 어려움을 겪고 있는 평범한 사람이다 .
120329,md_100610,독->한,"als sie aufwacht , ist sie doppelt so alt , de...","그녀가 깨어났을 때 , 그녀는 나이가 두 배나 많이 들었지만 , 여전히 10대처럼 ..."
120330,md_100611,독->한,"viele frauen stecken in der misere , und alle ...","많은 여성들이 불행 속에 살고 있고 , 그들 모두는 그들의 상황에서 벗어날 길을 찾..."


In [3]:
# 인코더의 입력 데이터인 src 데이터
raw_src_data = raw_data['원문(src)'].values.tolist()
# 디코더의 정답지 데이터인 tar 데이터
raw_tar_data = raw_data['번역문(tar)'].values.tolist()

In [4]:
from transformers import BertTokenizer

# 독일어 토크나이저 모델: dbmdz/bert-base-german-uncased
german_tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-german-uncased")
# 한국어 토크나이저 모델: kykim/bert-kor-base
korean_tokenizer = BertTokenizer.from_pretrained("kykim/bert-kor-base")

In [5]:
# 원문 토큰화 수행
tokenized_src_data = tokenize(raw_src_data, german_tokenizer, arch='Bert')
# 번역문 토큰화 수행
tokenized_tar_data = tokenize(raw_tar_data, korean_tokenizer, arch='Bert')

토큰화 진행 중: 100%|██████████| 120332/120332 [00:14<00:00, 8417.91it/s]


In [6]:
from sklearn.model_selection import train_test_split

# 훈련/검증/평가를 80%, 15%, 5%로 분할을 수행
# random_state -> 데이터셋을 내누는데 '재현성' 유지를 위해 넣음 -> 안넣어도 됨
# stratify -> y 클래스 비율을 알기 어렵기에 해당 항목은 없앰
src_train, src_etc, tar_train, tar_etc = train_test_split(
    tokenized_src_data, tokenized_tar_data, test_size=0.20,
    random_state=42
)

# 그 외 데이터셋을 반반으로 Val, Test로 나눔
src_val, src_test, tar_val, tar_test = train_test_split(
    src_etc, tar_etc, test_size=0.25,
    random_state=42
)

In [7]:
from collections import Counter

src_word_list = []
tar_word_list = []
# train항목을 워드 리스트에 입력
for src_sent, tar_sent in zip(src_train, tar_train):
    for word in src_sent:
        src_word_list.append(word)
    for word in tar_sent:
        tar_word_list.append(word)
# val항목을 워드 리스트에 입력
for src_sent, tar_sent in zip(src_train, tar_train):
    for word in src_sent:
        src_word_list.append(word)
    for word in tar_sent:
        tar_word_list.append(word)

# 단어와 해당 단어의 출몰 빈도를 함께 저장하는
# Counter 타입의 변수 생성
src_word_counts = Counter(src_word_list)
tar_word_counts = Counter(tar_word_list)

In [8]:
rare_th = 0 #희소단어의 등장 빈도를 결정하는 파라미터
# 희소단어 등장 빈도를 바탕으로 희소 단어를 배제하기 위해 준비 함수
print(f'---원문(src)에 대한 희소단어 분석---')
src_tot_vocab_cnt, src_rare_vocab_cnt = set_rare_vocab(src_word_counts, rare_th)
print(f'\n---번역문(tar)에 대한 희소단어 분석---')
tar_tot_vocab_cnt, tar_rare_vocab_cnt = set_rare_vocab(tar_word_counts, rare_th)

---원문(src)에 대한 희소단어 분석---

---번역문(tar)에 대한 희소단어 분석---


In [9]:
#등장 빈도가 높은 단어 순으로 정렬하기
src_vocab = sorted(src_word_counts, key=src_word_counts.get, reverse=True)
tar_vocab = sorted(tar_word_counts, key=tar_word_counts.get, reverse=True)

# 원문(src)에 대한 희소단어 배제 & 정렬 작업 수행
src_vocab_size = src_tot_vocab_cnt - src_rare_vocab_cnt
src_vocab = src_vocab[:src_vocab_size]
# 번역문(tar에 대한 희소단어 배제 & 정렬 작업 수행
tar_vocab_size = tar_tot_vocab_cnt - tar_rare_vocab_cnt
tar_vocab = tar_vocab[:tar_vocab_size]

In [10]:
# 스페셜 토큰 선언
spec_token = ['<PAD>', '<UNK>', '<SOS>', '<EOS>']
# 스페셜 토큰을 포함한 {단어:단어idx}의 딕셔너리 생성
print(f'---원문(src)에 대한 단어장 분석---')
src_to_idx, idx_to_src = set_word_to_idx(spec_token, src_vocab, 
                                         report=True)
print(f'\n---번역문(tar)에 대한 단어장 분석---')
tar_to_idx, idx_to_tar = set_word_to_idx(spec_token, tar_vocab, 
                                         report=True)

---원문(src)에 대한 단어장 분석---
단어집합(vocab)은 word_to_idx를 통해서
[단어 : idx]의 <class 'dict'>타입이 되고
스페셜 토큰 <PAD> <UNK> <SOS> <EOS> 을 포함하여
총 관리되는 단어 '22471' -> '22475'가 됨

---번역문(tar)에 대한 단어장 분석---
단어집합(vocab)은 word_to_idx를 통해서
[단어 : idx]의 <class 'dict'>타입이 되고
스페셜 토큰 <PAD> <UNK> <SOS> <EOS> 을 포함하여
총 관리되는 단어 '25697' -> '25701'가 됨


In [11]:
# 원문(src) 데이터셋의 정수 인코딩 수행
e_src_train = text_to_sequences(src_train, src_to_idx)
e_src_val = text_to_sequences(src_val, src_to_idx)
e_src_test = text_to_sequences(src_test, src_to_idx)

# 번역문(tar) 데이터셋의 정수 인코딩 수행
e_tar_train = text_to_sequences(tar_train, tar_to_idx)
e_tar_val = text_to_sequences(tar_val, tar_to_idx)
e_tar_test = text_to_sequences(tar_test, tar_to_idx)

In [12]:
# 번역문(tar)의 접두/접미에 SOS, EOS 토큰 추가
e_tar_train = prefix_suffix_token_insert(e_tar_train, spec_token)
e_tar_val = prefix_suffix_token_insert(e_tar_val, spec_token)
e_tar_test = prefix_suffix_token_insert(e_tar_test, spec_token)

In [13]:
# 원문(src)에 대한 문장패딩 하이퍼 파라미터 설정
src_seq_len = 60
set_sent_pad(e_src_train, src_seq_len)

# 번역문(tar)에 대한 문장패딩 하이퍼 파라미터 설정
tar_seq_len = 55
set_sent_pad(e_tar_train, tar_seq_len)

데이터셋 문장 길이가 60 이하 데이터 비율: 100.00%
데이터셋 문장 길이가 55 이하 데이터 비율: 100.00%


In [14]:
# 원문(src)의 문장 패딩(정수인코딩 완료)
padded_src_train = pad_seq_x(e_src_train, src_seq_len)
padded_src_val = pad_seq_x(e_src_val, src_seq_len)
padded_src_test = pad_seq_x(e_src_test, src_seq_len)

# 번역문(tar)의 문장 패딩(정수인코딩 완료)
padded_tar_train = pad_seq_x(e_tar_train, tar_seq_len)
padded_tar_val = pad_seq_x(e_tar_val, tar_seq_len)
padded_tar_test = pad_seq_x(e_tar_test, tar_seq_len)

In [15]:
print(f'---원문(src) 문장 패딩 결과---')
val_pad_shape(padded_src_train, '훈련')
val_pad_shape(padded_src_val, '검증')
val_pad_shape(padded_src_test, '평가')
print(f'\n---번역문(tar) 문장 패딩 결과---')
val_pad_shape(padded_tar_train, '훈련')
val_pad_shape(padded_tar_val, '검증')
val_pad_shape(padded_tar_test, '평가')

---원문(src) 문장 패딩 결과---
훈련용 정수(원핫)인코딩 shape: (96265, 60)
검증용 정수(원핫)인코딩 shape: (18050, 60)
평가용 정수(원핫)인코딩 shape: (6017, 60)

---번역문(tar) 문장 패딩 결과---
훈련용 정수(원핫)인코딩 shape: (96265, 55)
검증용 정수(원핫)인코딩 shape: (18050, 55)
평가용 정수(원핫)인코딩 shape: (6017, 55)


In [16]:
import torch
bs = 256 # Batch_size 하이퍼 파라미터

# 정수(원핫)인코딩 데이터를 데이터로더로 변환
trainloader = set_dataloader(padded_src_train, padded_tar_train, bs, 
                             content='훈련', report=True)
valloader = set_dataloader(padded_src_val, padded_tar_val, bs,
                           content='검증', report=True)
testloader = set_dataloader(padded_src_test, padded_tar_test, bs, 
                            content='평가', report=True)

훈련용 X(인코딩)데이터 크기: [96265, 60]
훈련용 Y(Label)데이터 크기: [96265, 55]
검증용 X(인코딩)데이터 크기: [18050, 60]
검증용 Y(Label)데이터 크기: [18050, 55]
평가용 X(인코딩)데이터 크기: [6017, 60]
평가용 Y(Label)데이터 크기: [6017, 55]


In [17]:
# 주요 하이퍼 파라미터 정리
src_VS = len(src_to_idx) # 원문 단어장 개수
tar_VS = len(tar_to_idx) # 번역문 단어장 개수
src_SL = src_seq_len # 원문의 문장 길이
# 번역문의 문장 길이를 디코더(생성)문장 길이로 쓰자
tar_SL = tar_seq_len # 번역문 문장 길이

EMB_DIM = 256 # 인코더/디코더의 임베딩 레이어 차원
# unit_dim은 인코더-디코더 사이의 히든레이어처럼 생각하는게 편함
UNIT_DIM = 256 # 인코더와 디코더의 rnn_out 차원값

NUM_Layers = 2 # 인코더/디코더의 셀은 2층으로 만들자
BI_DIR = False # 단방향으로만 학습

In [18]:
from tabulate import tabulate

# 출력할 데이터를 리스트 형식으로 준비
data = [
    ["원문 단어장 개수", f"{src_VS}개"],
    ["번역문 단어장 개수", f"{tar_VS}개"],
    ["원문 문장 길이", f"{src_SL}토큰"],
    ["번역문 문장 길이", f"{tar_SL}토큰"],
    ["원문/번역문 임베딩 차원", EMB_DIM],
    ["인코더-디코더 연결 차원", UNIT_DIM],
    ["셀 층 개수", f"{NUM_Layers}층"],
    ["양방향/단방향", "단방향 학습"],
]

# 표 형식으로 출력
print(tabulate(data, tablefmt="grid",
        headers=["하이퍼 파라미터", "값"]))

+-------------------------+-------------+
| 하이퍼 파라미터         | 값          |
+=========================+=============+
| 원문 단어장 개수        | 22475개     |
+-------------------------+-------------+
| 번역문 단어장 개수      | 25701개     |
+-------------------------+-------------+
| 원문 문장 길이          | 60토큰      |
+-------------------------+-------------+
| 번역문 문장 길이        | 55토큰      |
+-------------------------+-------------+
| 원문/번역문 임베딩 차원 | 256         |
+-------------------------+-------------+
| 인코더-디코더 연결 차원 | 256         |
+-------------------------+-------------+
| 셀 층 개수              | 2층         |
+-------------------------+-------------+
| 양방향/단방향           | 단방향 학습 |
+-------------------------+-------------+


In [19]:
import torch.nn as nn

class Encoder_LSTM(nn.Module):
    def __init__(self, src_vocab, src_emb_dim, rnn_dim, 
                 num_layer=1, bi=False):
        super(Encoder_LSTM, self).__init__()
        # <PAD>토큰의 인덱싱을 지정하면 해당 idx(0)은
        # word_vector을 만들 때 모두 0으로 채워지게 만들어준다.
        self.embed = nn.Embedding(src_vocab, src_emb_dim,
                                  padding_idx=0)
        self.lstm = nn.LSTM(input_size=src_emb_dim,#언어모델 입력차원
                            hidden_size=rnn_dim,   #언어모델 출력차원
                            num_layers=num_layer,  #언어모델 몇층?
                            bidirectional=bi,      #양방향학습 On?
                            batch_first=True)      #왠만하면 True
    
    def forward(self, x): # x의 차원 : (BS, src_seq_len)
        emb = self.embed(x) # (BS, src_seq_len, src_emb_dim)
        # 인코더에 양방향 학습을 적용한다
        # rnn_out : (bs, src_seq_len, hidden_dim * 2)
        # hidden : (num_layer * 2, bs, hidden_dim)
        rnn_out , (hidden, cell) = self.lstm(emb)
        #인코더의 출력은 context_vector
        return hidden, cell 

In [20]:
import torch.nn as nn

class Decoder_LSTM(nn.Module):
    def __init__(self, tar_vocab, tar_emb_dim, rnn_dim,
                 num_layer=1, bi=False):
        super(Decoder_LSTM, self).__init__()
        # <PAD>토큰의 인덱싱을 지정하면 해당 idx(0)은
        # word_vector을 만들 때 모두 0으로 채워지게 만들어준다.
        self.embed = nn.Embedding(tar_vocab, tar_emb_dim,
                                  padding_idx=0)
        self.lstm = nn.LSTM(input_size=tar_emb_dim,#언어모델 입력차원
                            hidden_size=rnn_dim,   #언어모델 출력차원
                            num_layers=num_layer,  #언어모델 몇층?
                            bidirectional=bi,      #양방향학습 On?
                            batch_first=True)      #왠만하면 True
        # 디코더의 출력은 정답(번역문)의 seq_len이 되게 해야함
        # 맞춰야 하는 클래스 개수는 정답지의 단어 개수임
        if bi : #양방향으로 학습시에는 FC 레이어 입력차원이 두배
            self.fc = nn.Linear(rnn_dim*2, tar_vocab)
        else:
            self.fc = nn.Linear(rnn_dim, tar_vocab)
    
    # 디코더는 인코더의 context_vector을 초기 hidden으로 입력받는다.
    def forward(self, x, hidden, cell): # x의 차원 : (BS, tar_seq_len)
        emb = self.embed(x) # (BS, tar_seq_len, tar_emb_dim)
        # 디코더에 양방향 학습을 적용한다
        # rnn_out : (bs, tar_seq_len, hidden_dim * 2)
        # hidden : (num_layer * 2, bs, hidden_dim)
        rnn_out, (hidden, cell) = self.lstm(emb, (hidden,cell))
        
        output = self.fc(rnn_out)
        # 최종 출력은 (bs, seq_len, tar_vocab)
        return output, hidden, cell

In [21]:
import torch.nn as nn

class Encoder_GRU(nn.Module):
    def __init__(self, src_vocab, src_emb_dim, rnn_dim, 
                 num_layer=1, bi=False):
        super(Encoder_GRU, self).__init__()
        # <PAD>토큰의 인덱싱을 지정하면 해당 idx(0)은
        # word_vector을 만들 때 모두 0으로 채워지게 만들어준다.
        self.embed = nn.Embedding(src_vocab, src_emb_dim,
                                  padding_idx=0)
        self.gru = nn.GRU(input_size=src_emb_dim,#언어모델 입력차원
                            hidden_size=rnn_dim,   #언어모델 출력차원
                            num_layers=num_layer,  #언어모델 몇층?
                            bidirectional=bi,      #양방향학습 On?
                            batch_first=True)      #왠만하면 True
    
    def forward(self, x): # x의 차원 : (BS, src_seq_len)
        emb = self.embed(x) # (BS, src_seq_len, src_emb_dim)
        # 인코더에 양방향 학습을 적용한다
        # rnn_out : (bs, src_seq_len, hidden_dim * 2)
        # hidden : (num_layer * 2, bs, hidden_dim)
        rnn_out , hidden = self.gru(emb)
        #인코더의 출력은 context_vector
        return hidden

In [22]:
import torch.nn as nn

class Decoder_GRU(nn.Module):
    def __init__(self, tar_vocab, tar_emb_dim, rnn_dim,
                 num_layer=1, bi=False):
        super(Decoder_GRU, self).__init__()
        # <PAD>토큰의 인덱싱을 지정하면 해당 idx(0)은
        # word_vector을 만들 때 모두 0으로 채워지게 만들어준다.
        self.embed = nn.Embedding(tar_vocab, tar_emb_dim,
                                  padding_idx=0)
        self.gru = nn.GRU(input_size=tar_emb_dim,#언어모델 입력차원
                            hidden_size=rnn_dim,   #언어모델 출력차원
                            num_layers=num_layer,  #언어모델 몇층?
                            bidirectional=bi,      #양방향학습 On?
                            batch_first=True)      #왠만하면 True
        # 디코더의 출력은 정답(번역문)의 seq_len이 되게 해야함
        # 맞춰야 하는 클래스 개수는 정답지의 단어 개수임
        if bi : #양방향으로 학습시에는 FC 레이어 입력차원이 두배
            self.fc = nn.Linear(rnn_dim*2, tar_vocab)
        else:
            self.fc = nn.Linear(rnn_dim, tar_vocab)
    
    # 디코더는 인코더의 context_vector을 초기 hidden으로 입력받는다.
    def forward(self, x, hidden): # x의 차원 : (BS, tar_seq_len)
        emb = self.embed(x) # (BS, tar_seq_len, tar_emb_dim)
        # 디코더에 양방향 학습을 적용한다
        # rnn_out : (bs, tar_seq_len, hidden_dim * 2)
        # hidden : (num_layer * 2, bs, hidden_dim)
        rnn_out, hidden = self.gru(emb, hidden)
        
        output = self.fc(rnn_out)
        # 최종 출력은 (bs, seq_len, tar_vocab)
        return output, hidden

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Seq2Seq_LSTM(nn.Module):
    def __init__(self, encoder, decoder, spec_token, max_len=None):
        super(Seq2Seq_LSTM, self).__init__()
        self.encode = encoder
        self.decode = decoder
        # 최대 디코딩 길이 설정
        self.max_len = max_len
        # 스페셜 토큰을 초기화에 입력하게 변경
        self.spec_token = spec_token

    def forward(self, src, tar=None, TF_ratio=1, BW=1):
        # 인코더의 출력 = context_vector
        context_h, context_c = self.encode(src)

        # 스페셜 토큰에서 SOS, EOS, PAD의 정수인코딩값 추출
        en_sos = self.spec_token.index('<SOS>')
        en_eos = self.spec_token.index('<EOS>')
        en_pad = self.spec_token.index('<PAD>')

        if tar is not None:
            # 배치사이즈, 연산위치 정보 추출(tar 기준으로)
            BS, tar_seq_len = tar.size() 
            device = tar.device

            # 디코더의 첫번째 토큰을 <SOS>에 (BS, 1)차원으로 생성
            # 이때 정답지(tar)의 맨 앞토큰은 <SOS>로 채워져 있으니 이를 이용한다.
            input_token = tar[:, 0].unsqueeze(1)
            outputs = [] # 출력 디코드 결과를 저장

            # 임의 난수를 (BS)차원으로 생성 후 TF_ratio비율정보를 받아서
            # 마스크 플래그로 변환, 이때 TF_ratio는 0~1 사이값
            # 1에 가까울수록 대부분의 Flag는 True가 되서 지도학습비율이 올라감
            TF_flag = torch.rand(BS, device=device) < TF_ratio

            # 토큰 단위로 예측이니 자가 회귀 방식임
            h, c = context_h, context_c

            # tar seq는 맨 처음 토큰을 <SOS>로 채웟으니 1번부터 시작
            for t in range(1, tar_seq_len):
                # 토큰단위로 입력이니 출력은 (bs, 1, tar_vocab)이다.
                out_tokens, h, c = self.decode(input_token, h, c)
                outputs.append(out_tokens)

                # 마스크 플래그가 True : 지도학습 방식으로 동작
                # 마스크 플래그가 False : 비지도학습-자가 회귀방식으로 동작
                input_token = torch.where(TF_flag.unsqueeze(1), 
                                            tar[:, t].unsqueeze(1),
                                            out_tokens.argmax(dim=-1))

            # 최종 출력 모양 조정 
            outputs = torch.cat(outputs, dim=1) # (BS, tar_seq_len-1, tar_vocab)
            # (BS, 1, tar_vocab) 차원의 sos 토큰 인덱스로 채워진 텐서를 만듬
            sos_tokens = torch.full((BS, 1, outputs.size(-1)), en_sos, device=device)
            # sos_tokens랑 outputs를 합쳐서 (BS, tar_seq_len, tar_vocab)가 되게 함
            outputs = torch.cat([sos_tokens, outputs], dim=1)
            return outputs # (BS, tar_seq_len, tar_vocab)
        
        else: # 정답지가 없는 평가모드
            # 배치사이즈, 연산위치 정보 추출(src 기준으로)
            BS, src_seq_len = src.size() 
            device = src.device
            # 최대 디코딩 길이 지정 안했으면 원문 seq_len을 쓰자
            if self.max_len is None:
                self.max_len = src_seq_len

            # 디코더의 첫 토큰을 <SOS>에 (BS,1)로 채우고 맨 앞에 Beam_width적용
            input_token = torch.full((BW, BS, 1), en_sos).to(device)
            # 디코더 첫 토큰에 대한 확률정보 생성 (BW, BS, 1)
            pred_probs = F.softmax(input_token.type(torch.float), dim=0)
            # 최종 출력 시퀀스는 Beam_search의 후보군 시퀀스를 다 내보내는 것으로 함
            # 후보 시퀀스를 다 내보낸 뒤 나중에 처리하는 것으로
            outputs = torch.full((BS, self.max_len, BW), en_pad).to(device)

            # <EOS>토큰을 각 beam search, batch_size에서 예측했으면 이를 감지하는 flag
            pad_mask_flag = torch.zeros((BW, BS), dtype=torch.bool, 
                                        device=device)

            # context_vector의 차원은 (num_layers, BS, hid_dim)임
            # 이것을 (BW, num_layers, BS, hid_dim) 4차원으로 반복복제로 늘림
            context_h = context_h.unsqueeze(0).repeat(BW, 1, 1, 1)
            context_c = context_c.unsqueeze(0).repeat(BW, 1, 1, 1) 

            # 토큰 단위로 예측이니 자가 회귀 방식임
            h, c = context_h, context_c

            for t in range(self.max_len):
                # 디코더에 입력하여 out_tokens를 만들어야 하니 입력 가능하게
                # BW * BS를 곱해서 3차원으로 축소
                input_token = input_token.view(BW*BS, -1) #(BW*BS, 1)
                h = h.view(h.size(1), BW*BS, h.size(3)) # (nL, BW*BS, hid_dim)
                c = c.view(c.size(1), BW*BS, c.size(3)) # (nL, BW*BS, hid_dim)
                
                # 출력된 out_tokens의 차원은 (BW*BS, 1, tar_vocab)이다.
                out_tokens, h, c = self.decode(input_token, h, c)

                # 원래 차원인 (BW, BS ,,)순으로 모두 복원
                out_tokens = out_tokens.view(BW, BS, 1, -1) # (BW, BS, 1, tar_vocab)
                # h = h.view(BW, h.size(0), BS, -1)   # (BW, nL, BS, hid_dim)
                # c = c.view(BW, c.size(0), BS, -1)   # (BW, nL, BS, hid_dim)
                
                # 모델이 예측한 토큰에 대한 확률정보 생성
                cur_probs = F.softmax(out_tokens, dim=-1) # (BW, BS, 1, tar_vocab)

                # cur_probs의 가장 높은 확률을 가진 BW(K)개 데이터 val, idx 추출
                # val, idx의 BW개를 선택한 차원정보 : (BW, BS, 1, BW)
                top_k_probs, top_k_idxs = torch.topk(input=cur_probs, k=BW, dim=-1)
                
                # pred_probs의 차원을 늘려서 (BW, BS, 1, 1)로 만든 다음
                # 브로드 캐스팅 방법으로 top_k_probs와 곱한 뒤 (BW, BS, 1, BW)
                # reshape를 적용하여 (BW*BW, BS, 1) 차원으로 변환
                scores = (pred_probs.unsqueeze(-1) * top_k_probs).view(BW*BW, BS, 1)
                # sorces 정보에 대해서 beam search로 BW개만큼의 데이터, idx 추출 (BW, BS, 1)
                _, top_sorce_idx = torch.topk(input=scores, k=BW, dim=0)

                # context vector 추적 및 추적정보를 바탕으로 갱신하기
                re_h = h.permute(1, 0, 2) # (BW * BS, nL, hid_dim)
                re_c = c.permute(1, 0, 2) # (BW * BS, nL, hid_dim)
                # (BW, BS, 1) -> (BW*BS, 1, 1) -> (BW*BS, nL, hid_dim)
                track_h_idx = (top_sorce_idx // BW).reshape(-1, 1, 1).expand(-1, re_h.size(1), re_h.size(2))
                track_c_idx = (top_sorce_idx // BW).reshape(-1, 1, 1).expand(-1, re_c.size(1), re_c.size(2))

                re_h = torch.gather(re_h, 0, track_h_idx) # (BW*BS, nL, hid_dim)
                re_c = torch.gather(re_c, 0, track_c_idx) # (BW*BS, nL, hid_dim)
                # (BW*BS, nL, hid_dim) -> (nL, BW*BS, hid_dim) -> (BW, nL, BS, hid_dim)
                h = re_h.permute(1, 0, 2).reshape(BW, h.size(0), BS, h.size(2))
                c = re_c.permute(1, 0, 2).reshape(BW, c.size(0), BS, c.size(2))

                # 이전토큰의 확률정보 pred_probs 업데이트
                # cur_probs에서 상위 BW개를 추출한 top_k_probs (BW, BS, 1, BW)를
                # (BW*BW, BS, 1)로 차원전환한 뒤 score의 BW개 idx 정보로 값 서치
                # 따라서 업데이트 된 pred_probs는 (BW, BS, 1)차원으로 정상적으로 업데이트
                pred_probs = torch.gather(top_k_probs.view(BW*BW, BS, 1), 0, top_sorce_idx)

                # 입력토큰은 top_sorce_idx 정보를 바탕으로 top_k_idxs의 인덱스 정보를 찾으면 됨
                input_token = torch.gather(top_k_idxs.view(BW*BW, BS, 1), 0, top_sorce_idx)

                # out_tokes를 최종 출력물을 outputs의 t번째 seq에 덮어쓰기
                outputs[:, t, :] = input_token.squeeze(-1).transpose(0, 1)  # (BS, max_len, BW)

                # 업데이트한 input_token이 <EOS>토큰의 idx를 예측햇는지 검토
                eos_indices = (input_token.squeeze(1) == en_eos)
                # OR연산을 통해 pad_mask_flag가 <EOS>토큰의 idx를 예측했으면 TRUE로 전환한다
                pad_mask_flag = pad_mask_flag | eos_indices

                # 모든 샘플이 EOS예측
                if pad_mask_flag.all():
                    break
            
            return outputs # (BS, max_len, BW)

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Seq2Seq_GRU(nn.Module):
    def __init__(self, encoder, decoder, spec_token, max_len=None):
        super(Seq2Seq_GRU, self).__init__()
        self.encode = encoder
        self.decode = decoder
        # 최대 디코딩 길이 설정
        self.max_len = max_len
        # 스페셜 토큰을 초기화에 입력하게 변경
        self.spec_token = spec_token

    def forward(self, src, tar=None, TF_ratio=1, BW=1):
        # 인코더의 출력 = context_vector
        context_h = self.encode(src)

        # 스페셜 토큰에서 SOS, EOS, PAD의 정수인코딩값 추출
        en_sos = self.spec_token.index('<SOS>')
        en_eos = self.spec_token.index('<EOS>')
        en_pad = self.spec_token.index('<PAD>')

        if tar is not None:
            # 배치사이즈, 연산위치 정보 추출(tar 기준으로)
            BS, tar_seq_len = tar.size() 
            device = tar.device

            # 디코더의 첫번째 토큰을 <SOS>에 (BS, 1)차원으로 생성
            # 이때 정답지(tar)의 맨 앞토큰은 <SOS>로 채워져 있으니 이를 이용한다.
            input_token = tar[:, 0].unsqueeze(1)
            outputs = [] # 출력 디코드 결과를 저장

            # 임의 난수를 (BS)차원으로 생성 후 TF_ratio비율정보를 받아서
            # 마스크 플래그로 변환, 이때 TF_ratio는 0~1 사이값
            # 1에 가까울수록 대부분의 Flag는 True가 되서 지도학습비율이 올라감
            TF_flag = torch.rand(BS, device=device) < TF_ratio

            # 토큰 단위로 예측이니 자가 회귀 방식임
            h = context_h

            # tar seq는 맨 처음 토큰을 <SOS>로 채웟으니 1번부터 시작
            for t in range(1, tar_seq_len):
                # 토큰단위로 입력이니 출력은 (bs, 1, tar_vocab)이다.
                out_tokens, h = self.decode(input_token, h)
                outputs.append(out_tokens)

                # 마스크 플래그가 True : 지도학습 방식으로 동작
                # 마스크 플래그가 False : 비지도학습-자가 회귀방식으로 동작
                input_token = torch.where(TF_flag.unsqueeze(1), 
                                            tar[:, t].unsqueeze(1),
                                            out_tokens.argmax(dim=-1))

            # 최종 출력 모양 조정 
            outputs = torch.cat(outputs, dim=1) # (BS, tar_seq_len-1, tar_vocab)
            # (BS, 1, tar_vocab) 차원의 sos 토큰 인덱스로 채워진 텐서를 만듬
            sos_tokens = torch.full((BS, 1, outputs.size(-1)), en_sos, device=device)
            # sos_tokens랑 outputs를 합쳐서 (BS, tar_seq_len, tar_vocab)가 되게 함
            outputs = torch.cat([sos_tokens, outputs], dim=1)
            return outputs # (BS, tar_seq_len, tar_vocab)
        
        else: # 정답지가 없는 평가모드
            # 배치사이즈, 연산위치 정보 추출(src 기준으로)
            BS, src_seq_len = src.size() 
            device = src.device
            # 최대 디코딩 길이 지정 안했으면 원문 seq_len을 쓰자
            if self.max_len is None:
                self.max_len = src_seq_len

            # 디코더의 첫 토큰을 <SOS>에 (BS,1)로 채우고 맨 앞에 Beam_width적용
            input_token = torch.full((BW, BS, 1), en_sos).to(device)
            # 디코더 첫 토큰에 대한 확률정보 생성 (BW, BS, 1)
            pred_probs = F.softmax(input_token.type(torch.float), dim=0)
            # 최종 출력 시퀀스는 Beam_search의 후보군 시퀀스를 다 내보내는 것으로 함
            # 후보 시퀀스를 다 내보낸 뒤 나중에 처리하는 것으로
            outputs = torch.full((BS, self.max_len, BW), en_pad).to(device)

            # <EOS>토큰을 각 beam search, batch_size에서 예측했으면 이를 감지하는 flag
            pad_mask_flag = torch.zeros((BW, BS), dtype=torch.bool, 
                                        device=device)

            # context_vector의 차원은 (num_layers, BS, hid_dim)임
            # 이것을 (BW, num_layers, BS, hid_dim) 4차원으로 반복복제로 늘림
            context_h = context_h.unsqueeze(0).repeat(BW, 1, 1, 1)

            # 토큰 단위로 예측이니 자가 회귀 방식임
            h = context_h

            for t in range(self.max_len):
                # 디코더에 입력하여 out_tokens를 만들어야 하니 입력 가능하게
                # BW * BS를 곱해서 3차원으로 축소
                input_token = input_token.view(BW*BS, -1) #(BW*BS, 1)
                h = h.view(h.size(1), BW*BS, h.size(3)) # (nL, BW*BS, hid_dim)
                
                # 출력된 out_tokens의 차원은 (BW*BS, 1, tar_vocab)이다.
                out_tokens, h = self.decode(input_token, h)
                # h = h.view(BW, h.size(0), BS, -1)   # (BW, nL, BS, hid_dim)

                # 원래 차원인 (BW, BS ,,)순으로 모두 복원
                out_tokens = out_tokens.view(BW, BS, 1, -1) # (BW, BS, 1, tar_vocab)
                # 모델이 예측한 토큰에 대한 확률정보 생성
                cur_probs = F.softmax(out_tokens, dim=-1) # (BW, BS, 1, tar_vocab)

                # cur_probs의 가장 높은 확률을 가진 BW(K)개 데이터 val, idx 추출
                # val, idx의 BW개를 선택한 차원정보 : (BW, BS, 1, BW)
                top_k_probs, top_k_idxs = torch.topk(input=cur_probs, k=BW, dim=-1)
                
                # pred_probs의 차원을 늘려서 (BW, BS, 1, 1)로 만든 다음
                # 브로드 캐스팅 방법으로 top_k_probs와 곱한 뒤 (BW, BS, 1, BW)
                # reshape를 적용하여 (BW*BW, BS, 1) 차원으로 변환
                scores = (pred_probs.unsqueeze(-1) * top_k_probs).view(BW*BW, BS, 1)
                # sorces 정보에 대해서 beam search로 BW개만큼의 데이터, idx 추출 (BW, BS, 1)
                _, top_sorce_idx = torch.topk(input=scores, k=BW, dim=0)

                # context vector 추적 및 추적정보를 바탕으로 갱신하기
                re_h = h.permute(1, 0, 2) # (BW * BS, nL, hid_dim)
                # (BW, BS, 1) -> (BW*BS, 1, 1) -> (BW*BS, nL, hid_dim)
                track_h_idx = (top_sorce_idx // BW).reshape(-1, 1, 1).expand(-1, re_h.size(1), re_h.size(2))

                re_h = torch.gather(re_h, 0, track_h_idx) # (BW*BS, nL, hid_dim)
                # (BW*BS, nL, hid_dim) -> (nL, BW*BS, hid_dim) -> (BW, nL, BS, hid_dim)
                h = re_h.permute(1, 0, 2).reshape(BW, h.size(0), BS, h.size(2))


                # 이전토큰의 확률정보 pred_probs 업데이트
                # cur_probs에서 상위 BW개를 추출한 top_k_probs (BW, BS, 1, BW)를
                # (BW*BW, BS, 1)로 차원전환한 뒤 score의 BW개 idx 정보로 값 서치
                # 따라서 업데이트 된 pred_probs는 (BW, BS, 1)차원으로 정상적으로 업데이트
                pred_probs = torch.gather(top_k_probs.view(BW*BW, BS, 1), 0, top_sorce_idx)

                # 입력토큰은 top_sorce_idx 정보를 바탕으로 top_k_idxs의 인덱스 정보를 찾으면 됨
                input_token = torch.gather(top_k_idxs.view(BW*BW, BS, 1), 0, top_sorce_idx)

                # out_tokes를 최종 출력물을 outputs의 t번째 seq에 덮어쓰기
                outputs[:, t, :] = input_token.squeeze(-1).transpose(0, 1)  # (BS, max_len, BW)

                # 업데이트한 input_token이 <EOS>토큰의 idx를 예측햇는지 검토
                eos_indices = (input_token.squeeze(1) == en_eos)
                # OR연산을 통해 pad_mask_flag가 <EOS>토큰의 idx를 예측했으면 TRUE로 전환한다
                pad_mask_flag = pad_mask_flag | eos_indices

                # 모든 샘플이 EOS예측
                if pad_mask_flag.all():
                    break

            return outputs # (BS, max_len, BW)

In [25]:
# 학습 실험 조건을 구분하기 위한 키
model_key = ['LSTM', 'GRU']
metrics_key = ['Loss', '정확도']

In [26]:
# Seq2Seq의 LSTM버전 인스턴스화
encoder_lstm = Encoder_LSTM(src_VS, EMB_DIM, UNIT_DIM, 
                            num_layer=NUM_Layers, bi=BI_DIR)
decoder_lstm = Decoder_LSTM(tar_VS, EMB_DIM, UNIT_DIM,
                            num_layer=NUM_Layers, bi=BI_DIR)
Translater_lstm = Seq2Seq_LSTM(encoder_lstm, decoder_lstm, 
                               spec_token=spec_token, max_len=tar_SL)

# Seq2Seq의 GRU버전 인스턴스화
encoder_gru = Encoder_GRU(src_VS, EMB_DIM, UNIT_DIM, 
                            num_layer=NUM_Layers, bi=BI_DIR)
decoder_gru = Decoder_GRU(tar_VS, EMB_DIM, UNIT_DIM,
                            num_layer=NUM_Layers, bi=BI_DIR)
Translater_gru = Seq2Seq_GRU(encoder_gru, decoder_gru, 
                             spec_token=spec_token, max_len=tar_SL)

In [27]:
# 학습된 모델 저장
path = {} #모델별 경로명 저장
for mk in model_key:
    path[mk] = f'Seq2Seq_{mk}.pth'
    # torch.save(models[mk].state_dict(), path[mk])

In [28]:
# 저장된 모델 불러오기
load_model = {
    'LSTM': Translater_lstm,  # LSTM 모델 인스턴스 생성
    'GRU': Translater_gru     # GRU 모델 인스턴스 생성
}
for mk in model_key:
    load_model[mk].load_state_dict(torch.load(path[mk], weights_only=True))
    #추론기는 CPU에서 돌리자
    load_model[mk] = load_model[mk].to('cpu')

In [29]:
import random
# 테스트 데이터셋에서 샘플을 추출
# 전체 테스트 데이터 개수정보를 추출
num_test = padded_src_test.shape[0]
sample_epoch = 5 #추출할 샘플 개수 정의
indices = random.sample(range(num_test), sample_epoch)

# 추출한 샘플번호를 바탕으로 Test 데이터셋에서 무작위 추출
S_src_test = padded_src_test[indices]
S_tar_test = padded_tar_test[indices]

# 원문 데이터만 텐서 자료형으로 변환
TS_src_test = torch.tensor(S_src_test, dtype=torch.long)

In [30]:
from Seq_trainer import *

# 추론 결과를 저장할 딕셔너리
tar_dict = {key: []  for key in model_key}

BW = 3 #Beam Search의 Search Space 계수(Beam_width)값

for key in model_key:
    print(f"\n--현재 추론 조건: [Seq2Seq_{key}]--") # 조건에 맞는 실험시작
    for idx in tqdm(range(sample_epoch)): #추론 에포크별 추론 시작
        # 입력되는 원문 차원을 (1, src_seq_len)으로 만들기 위한 코드
        iter_src_data = TS_src_test[idx].unsqueeze(0)
        # 모델 추론 -> 추론결과는 (bs, max_len, BW) ndarray타입임
        # BW 옵션이 1 -> 사실상 Greedy Search랑 같은 결과
        # BW 옵션이 1 이상 -> 제대로된 Beam Search 작업수행
        tar_infer_doc = model_inference(load_model[key], 
                                        iter_src_data, BW=BW)
        tar_dict[key].append(tar_infer_doc)


--현재 추론 조건: [Seq2Seq_LSTM]--


100%|██████████| 5/5 [00:00<00:00, 17.21it/s]



--현재 추론 조건: [Seq2Seq_GRU]--


100%|██████████| 5/5 [00:00<00:00, 16.92it/s]


In [31]:
idx_list = range(1, sample_epoch+1)
for idx, src, pred_lstm, pred_gru, tar in zip(idx_list, S_src_test, 
                                              tar_dict['LSTM'], 
                                              tar_dict['GRU'], 
                                              S_tar_test):
    # 원문, 모델번역문_1, 모델번역문_2, 정답번역문 순으로 디코딩

    decode_src = Translater_post_processor(src, idx_to_src, spec_token)
    
    de_pred_lstm_list = [] #Beam_search를 수행함으로 인한 후보군 데이터를 저장하는 리스트 
    de_pred_gru_list = []  #Beam_search를 수행함으로 인한 후보군 데이터를 저장하는 리스트

    for BW_i in range(BW):
        de_pred_lstm = Translater_post_processor(pred_lstm[:, BW_i], idx_to_tar, spec_token)
        de_pred_gru = Translater_post_processor(pred_gru[:, BW_i], idx_to_tar, spec_token)
        
        de_pred_lstm_list.append(de_pred_lstm)
        de_pred_gru_list.append(de_pred_gru)

    decode_tar = Translater_post_processor(tar, idx_to_tar, spec_token)

    print(f"{idx}번째 번역 결과 확인")
    print(f"원문(src) : {decode_src}", end='\n\n')

    for BW_i in range(BW):
        print(f"seqLSTM_{BW_i}_번역 : {de_pred_lstm_list[BW_i]}")
        print(f"seq-GRU_{BW_i}_번역 : {de_pred_gru_list[BW_i]}")

    print(f"\n번역문(tar) : {decode_tar}")
    print("==============================\n")

1번째 번역 결과 확인
원문(src) : dann nehmen sie die jungen weg .

seqLSTM_0_번역 : 그런 다음 그들은 함께 일했습니다 . . .
seq-GRU_0_번역 : 그러면0살되어 있습니다
seqLSTM_1_번역 : 그런 다음 그들은 함께 일했습니다 . 있습니다 . .
seq-GRU_1_번역 : 그러면 그들은명의됩니다 . .
seqLSTM_2_번역 : 처리가 그녀는 약속을 돌하게 살고 있습니다 있습니다 . 있습니다
seq-GRU_2_번역 : 00 그들은들되었다 .

번역문(tar) : 그런 다음 그들은 새끼들을 데려갑니다 .

2번째 번역 결과 확인
원문(src) : seit geraumer zeit wird sie von ihrem mann vernachlassigt und genau das beabsichtigt sie mittels ihrer wurzigen kochkunst zu andern .

seqLSTM_0_번역 : 형식의 접목 처리가 여유회사의 모의 그녀의 인내를 그들의 그들의 함께 그들이놓을 목표를 대해하고 있습니다 .는 .
seq-GRU_0_번역 : 그녀의 주요 번째 주요 순간 , 즉 , 없는 동안 ,하는 것은 , 여성의 그들의 오랜적 재능으로 아니라 , 되돌아이 .으로 결정 .
seqLSTM_1_번역 : 분배 처리가선거 그녀의 마지막에 대한 그들의 이익을 위해 그들은과 그들의 그들은 놓는 것에 달성 이야기하고 있습니다 . .자 .는 것입니다 . .
seq-GRU_1_번역 : 그녀의 주요 순간 방향점은 , 그러나 정기적으로 겨울 시대에 회복흔든 , 의사는 그들의 음악 보석이고 집주인에게흙었습니다 되었습니다 .도 데려했습니다 .
seqLSTM_2_번역 : 데스는 이미 처리가 출생을 그녀가 봤던 감정을심을 그들 삶을 영원히 바꿔 놓 신호들을 이야기하는 것을 발견합니다 )고 . .는
seq-GRU_2_번역 : 보트 첫 순간 중에는 진 여성이 혼란이 진행되는과 뒤 아이도는 리 루이스들은 폭 주요 시간 수준을이의 진

In [38]:
import numpy as np
from collections import Counter
from scipy.special import softmax

def BLEU_Score(candidate, reference, p_list=None, ignore_idx=0):
    if p_list is None:
        p_list = [0.25, 0.25, 0.25, 0.25]  # 기본적으로 균등 가중치 설정
    
    def n_gram_precision(candidate, reference, n, ignore_idx):
        candidate_n_grams = [tuple(candidate[i:i+n]) 
                             for i in range(len(candidate) - n + 1) 
                                if ignore_idx not in candidate[i:i+n]]
        reference_n_grams = [tuple(reference[i:i+n]) 
                             for i in range(len(reference) - n + 1) 
                                if ignore_idx not in reference[i:i+n]]
        
        candidate_counter = Counter(candidate_n_grams)
        reference_counter = Counter(reference_n_grams)
        
        overlap = sum(min(candidate_counter[ng], reference_counter[ng]) 
                                for ng in candidate_counter)
        total = sum(candidate_counter.values())
        
        if total == 0:
            return 0
        else:
            return overlap / total


    best_bleu_score = 0
    best_candidate = None
    bleu_score_list = []

    for i in range(candidate.shape[1]):  # BW 개수만큼 후보군을 반복
        current_candidate = candidate[:, i]
        precisions = []
        
        for n in range(1, 5):  # 1-gram, 2-gram, 3-gram, 4-gram
            p_n = n_gram_precision(current_candidate, reference, n, ignore_idx)
            # Smoothing: precision이 0인 경우 작은 값(예: 1e-9)으로 대체
            precisions.append(p_n if p_n > 0 else 1e-9)
        
        log_precisions = [p_list[n-1] * np.log(p) for n, p in 
                            enumerate(precisions, start=1)]
        bleu_score = np.exp(sum(log_precisions))

        bleu_score_list.append(bleu_score*(10^20))
        

    # bleu_score_result의 값이 원체 작아서 softmax 처리
    bleu_score_list = softmax(bleu_score_list)

    # 가장 높은 BLEU score를 가진 후보군 선택
    best_idx = np.argmax(bleu_score_list)
    best_bleu_score = bleu_score_list[best_idx]
    best_candidate = candidate[:, best_idx]
    
    bleu_dict = {
        '최고BLUE점수': f'{best_bleu_score * 100:.2f}%',
        'Best_y예측': best_candidate
    }
    
    return bleu_dict

In [45]:
# BLEU score를 적용하여 원문 / 예측구문 / 번역문 결과 확인
idx_list = range(1, sample_epoch+1)
for idx, src, pred_lstm, pred_gru, tar in zip(idx_list, S_src_test, 
                                              tar_dict['LSTM'], 
                                              tar_dict['GRU'], 
                                              S_tar_test):
    # 원문, 모델번역문_1, 모델번역문_2, 정답번역문 순으로 디코딩
    decode_src = Translater_post_processor(src, idx_to_src, spec_token)

    # 모델 예측 결과물에 대하여 BLEU 스코어 및 Best_y_pred 추출
    BLEU_LSTM = BLEU_Score(pred_lstm, tar)
    BLEU_LSTM_score = BLEU_LSTM['최고BLUE점수']
    b_de_pred_lstm = Translater_post_processor(BLEU_LSTM['Best_y예측'], idx_to_tar, spec_token)
    BLEU_GRU = BLEU_Score(pred_gru, tar)
    BLEU_GRU_score = BLEU_GRU['최고BLUE점수']
    b_de_pred_gru = Translater_post_processor(BLEU_GRU['Best_y예측'], idx_to_tar, spec_token)

    decode_tar = Translater_post_processor(tar, idx_to_tar, spec_token)

    print(f"{idx}번째 번역 결과 확인")
    print(f"원문(src) : {decode_src}", end='\n')

    print(f"seqLSTM_점수: {BLEU_LSTM_score}\nseqLSTM_번역: {b_de_pred_lstm}")
    print(f"seqGRU_점수: {BLEU_GRU_score}\nseqGRU_번역: {b_de_pred_gru}")

    print(f"번역문(tar) : {decode_tar}")
    print("==============================\n")

1번째 번역 결과 확인
원문(src) : dann nehmen sie die jungen weg .
seqLSTM_점수: 33.94%
seqLSTM_번역: 그런 다음 그들은 함께 일했습니다 . . .
seqGRU_점수: 33.34%
seqGRU_번역: 그러면 그들은명의됩니다 . .
번역문(tar) : 그런 다음 그들은 새끼들을 데려갑니다 .

2번째 번역 결과 확인
원문(src) : seit geraumer zeit wird sie von ihrem mann vernachlassigt und genau das beabsichtigt sie mittels ihrer wurzigen kochkunst zu andern .
seqLSTM_점수: 33.33%
seqLSTM_번역: 형식의 접목 처리가 여유회사의 모의 그녀의 인내를 그들의 그들의 함께 그들이놓을 목표를 대해하고 있습니다 .는 .
seqGRU_점수: 33.33%
seqGRU_번역: 그녀의 주요 번째 주요 순간 , 즉 , 없는 동안 ,하는 것은 , 여성의 그들의 오랜적 재능으로 아니라 , 되돌아이 .으로 결정 .
번역문(tar) : 오랫동안 남편에게 외면당했던 그녀는 향기로운 요리를 통해 그것을 변화시키려는 것이다 .

3번째 번역 결과 확인
원문(src) : biagini suchte 45 minuten lang nach ihm , bevor der wal nur etwa 180 meter von seinem bug entfernt auftauchte .
seqLSTM_점수: 33.33%
seqLSTM_번역: 분배 처리가 10억 . . .
seqGRU_점수: 33.34%
seqGRU_번역: 약방구가 2000년 전에 대부분의 가이드 , 터널 표시를서 10분해 왔습니다 . .에 가장
번역문(tar) : 비아기니는 고래가 뱃서 불과 180m 떨어진 지점에 나타나기 전까지 45분 동안 고래를 찾았습니다 .

4번째 번역 결과 확인
원문(src) : es ist die energie , die außergewohn